# CSC 480-F25 Lab 4: Agentic Monte Carlo Search
### Authors:
***[Your Name Here]***

California Polytechnic State University, San Luis Obispo;
Computer Science & Software Engineering Department

### Overview
This lab focuses on:
*   Understanding and implementing Monte Carlo Search for a complex puzzle (NYT Spelling Bee).
*   Improving the random simulation (rollout) process by designing a smarter heuristic.
*   Decomposing the Monte Carlo Search algorithm into a multi-agent system using AutoGen.
*   Specifying the communication protocols (MCP and A2A) for agent collaboration.
*   Analyzing the trade-offs between conventional and agentic implementations of search algorithms.

**NOTE:** The Spelling Bee problem definition and a baseline Monte Carlo search function are provided for you. Your work is to improve the heuristic (Part 1) and then re-implement the search using an agentic architecture (Part 2).

### Learning Objectives
By the end of this lab, you will be able to:
*   Understand the principles of Monte Carlo Search and its trade-offs compared to systematic search algorithms.
*   Implement a conventional Monte Carlo Search algorithm by adapting a generalized search framework.
*   Design and implement an agentic system to perform and evaluate the random simulations (rollouts).
*   Analyze how the number of simulations impacts solution quality and performance.
*   Specify the communication patterns (MCP/A2A) required for agents to collaboratively execute and aggregate the results.

### Environment Setup
Install the required packages for AutoGen and other utilities. This setup is similar to previous labs.

In [ ]:
%pip install "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"

In [ ]:
import os
import asyncio
import random
from collections import Counter

# Import AutoGen classes, similar to Lab 2 and 3
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

### Azure OpenAI Configuration
Set up your Azure OpenAI client configuration. **Remember to replace the placeholder values with your actual deployment details** as you did in previous labs.

In [ ]:
# Configure your Azure OpenAI client
azure_deployment = "your-deployment-name"
api_version = "2024-12-01-preview"
azure_endpoint = "your-azure-endpoint"  # e.g., "https://your-resource-name.openai.azure.com/"

# Ensure your API key is set as an environment variable for security
api_key = os.getenv("AZURE_SUBSCRIPTION_KEY")

if not api_key:
    raise ValueError("AZURE_SUBSCRIPTION_KEY environment variable not set.")

client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_deployment,
    model="gpt-5-mini",
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
)

### The NY Times Spelling Bee Problem
The following class defines the Spelling Bee problem. It includes methods to check for valid words, calculate scores, and determine if a state is a goal state. You will not need to modify this class, but you should understand its methods. This setup is similar to the problem definition in Lab 3.

In [ ]:
class SpellingBeeProblem:
    """Defines the NYT Spelling Bee puzzle.
    A state is represented by the current word being built (e.g., 'APPLE').
    An action is appending a valid letter.
    """

    def __init__(self, letters, required_letter, dictionary):
        self.letters = set(letters)
        self.required_letter = required_letter
        self.dictionary = dictionary
        self.initial_state = ""

    def get_successor_states(self, state):
        """Generate all possible next states (words) by appending one letter."""
        successors = []
        for letter in self.letters:
            successors.append(state + letter)
        return successors

    def is_valid_word(self, word):
        """Check if a word is a valid solution."""
        return (
            len(word) >= 4
            and self.required_letter in word
            and word.lower() in self.dictionary
        )

    def get_score(self, word):
        """Calculate the score for a valid word."""
        if not self.is_valid_word(word):
            return 0
        score = len(word)
        if len(word) == 4:
            score = 1
        if set(word) == self.letters:
            score += 7  # Pangram bonus
        return score


# For this lab, we'll use a small, simple dictionary for demonstration purposes.
simple_dictionary = {"apple", "apply", "appeal", "pale", "peel", "plea", "leap", "app"}
puzzle = SpellingBeeProblem(
    letters={"A", "P", "L", "E", "Y"}, required_letter="A", dictionary=simple_dictionary
)

## Part 1: Conventional Monte Carlo Search and Heuristic Improvement

In this part, you will work with a conventional implementation of Monte Carlo Search. The core idea is to evaluate a potential move (i.e., the next state) by running many random simulations (rollouts) from that state and averaging the outcomes.

The provided `run_simulation` function uses a **simple heuristic**: it picks the next letter with uniform random probability. Your task is to improve this.

In [ ]:
def run_simulation(
    problem: SpellingBeeProblem,
    start_state: str,
    max_depth: int = 10,
    heuristic_fn=None,
):
    """Runs one random rollout from a given state and returns the score."""
    current_state = start_state
    for _ in range(max_depth):
        if problem.is_valid_word(current_state):
            return problem.get_score(current_state)

        possible_next_letters = list(problem.letters)
        if not possible_next_letters:
            break

        if heuristic_fn:
            # Use the heuristic to pick the next letter
            next_letter = heuristic_fn(current_state, possible_next_letters)
        else:
            # Default: simple uniform random choice
            next_letter = random.choice(possible_next_letters)

        current_state += next_letter

    return problem.get_score(current_state)


def monte_carlo_search(
    problem: SpellingBeeProblem,
    current_state: str,
    num_simulations: int = 100,
    heuristic_fn=None,
):
    """Evaluates successor states using Monte Carlo rollouts and chooses the best one."""
    successors = problem.get_successor_states(current_state)
    best_successor = None
    best_avg_score = -1

    for successor in successors:
        total_score = 0
        for _ in range(num_simulations):
            total_score += run_simulation(problem, successor, heuristic_fn=heuristic_fn)

        avg_score = total_score / num_simulations
        print(f"Successor '{successor}' has average score: {avg_score:.2f}")

        if avg_score > best_avg_score:
            best_avg_score = avg_score
            best_successor = successor

    return best_successor, best_avg_score

### Your Task (Part 1): Implement an Improved Heuristic
Create a new heuristic function `improved_heuristic`. This function should be "smarter" than a simple random choice. It takes the `current_word` and a list of `possible_letters` and returns the chosen next letter.

**Ideas for your heuristic:**
*   **Letter Frequency:** Prioritize letters that are more common in English.
*   **Avoid Repetition:** Penalize adding a letter that is already in the word.
*   **Seek the Required Letter:** If the required letter isn't in the word yet, give it a higher probability.
*   **Pangram Seeking:** Prioritize using all unique letters from the puzzle.

In [ ]:
from typing import List

# A simple letter frequency map for English (you can find more detailed ones online)
ENGLISH_LETTER_FREQ = {
    "E": 12.7,
    "T": 9.1,
    "A": 8.2,
    "O": 7.5,
    "I": 7.0,
    "N": 6.7,
    "S": 6.3,
    "H": 6.1,
    "R": 6.0,
    "D": 4.3,
    "L": 4.0,
    "C": 2.8,
    "U": 2.8,
    "M": 2.4,
    "W": 2.4,
    "F": 2.2,
    "G": 2.0,
    "Y": 2.0,
    "P": 1.9,
    "B": 1.5,
    "V": 1.0,
    "K": 0.8,
    "J": 0.2,
    "X": 0.2,
    "Q": 0.1,
    "Z": 0.1,
}


def improved_heuristic(current_word, possible_letters: List[str]) -> str:
    """
    Implement your smarter heuristic here.
    This function should return a single chosen letter from `possible_letters`.
    """
    # --- YOUR CODE GOES HERE ---
    # This function uses the probabilities from ENGLISH_LETTER_FREQ to make a weighted random choice.
    # You should try to improve it more. Replace this!

    # For example, create a list of weights for each possible letter
    weights = [
        ENGLISH_LETTER_FREQ.get(letter.upper(), 1.0) for letter in possible_letters
    ]

    # Use random.choices to pick a letter based on the weights
    chosen_letter = random.choices(possible_letters, weights=weights, k=1)[0]

    return chosen_letter
    # --- END OF YOUR CODE ---

### Run and Compare Heuristics
Now, run the Monte Carlo search with both the baseline (no heuristic) and your improved heuristic to see the difference.

In [ ]:
print("--- Running with Baseline Heuristic (Uniform Random) ---")
best_move_base, best_score_base = monte_carlo_search(
    puzzle, current_state="", num_simulations=500
)
print(
    f"\nBest next letter with baseline heuristic: '{best_move_base}' with score {best_score_base:.2f}\n"
)

print("--- Running with Improved Heuristic ---")
best_move_improved, best_score_improved = monte_carlo_search(
    puzzle, current_state="", num_simulations=500, heuristic_fn=improved_heuristic
)
print(
    f"\nBest next letter with improved heuristic: '{best_move_improved}' with score {best_score_improved:.2f}"
)

### Reflection (Part 1)
*Write a few sentences here reflecting on your heuristic. Did it perform better than the baseline? Why or why not? What other factors could you incorporate to make it even better?*

## Part 2: Agentic Implementation of Monte Carlo Search

Now, you will refactor the Monte Carlo search into a multi-agent system using AutoGen. This exercise is similar to the task decomposition in Lab 2, where you assign specialized roles to different agents.

### Your Task (Part 2): Design and Implement the Agentic System

You need to create and configure a team of agents to perform the search. We suggest the following roles, which follow a **Manager-Worker** pattern:

1.  **Orchestrator Agent (Manager)**: Manages the overall process. It identifies the possible next moves (successors), asks for them to be evaluated, and then chooses the best one.
2.  **Simulator Agent (Worker)**: Its only job is to run a single random simulation for a given state using your `improved_heuristic`.
3.  **Aggregator Agent**: This agent receives multiple simulation scores for a single successor state and calculates the average.

First, define the system prompts for these agents.

In [ ]:
# --- YOUR CODE GOES HERE: Define system messages for each agent ---

orchestrator_system_message = """
You are the Orchestrator.
Your job is to find the best next letter in the Spelling Bee puzzle.
1. You will be given the current word and the list of possible next letters.
2. For each appropriate single-letter successor word, ask the Simulator to evaluate it.
3. The Simulator will produce the average score (higher is better) for each word you ask it to evaluate.
4. You may repeat steps 2-3 as many times as you like. Once you decide on a successor, say "SUCCESSOR: <letter>."
"""

simulator_system_message = """
You are a simulation agent.
For every word the Orchestrator asks you to evaluate, you will run an appropriate number of simulations using the \
provided `run_simulations_tool` which returns the average score to report back.
"""

# --- END OF YOUR CODE ---

### Registering Tools for Agents (MCP)

To allow agents to perform actions, we need to register functions as tools they can call. This is an example of the Model Context Protocol (MCP), where we define a clear schema for how the agent can interact with its environment.

In [ ]:
# This is the function the Simulator agent will call.
def run_simulations_tool(word: str, num_simulations: int) -> float:
    """Runs num_simulations simulations for a given word and returns the average score."""

    # We use the improved heuristic from Part 1
    scores = [
        run_simulation(puzzle, start_state=word, heuristic_fn=improved_heuristic)
        for _ in range(num_simulations)
    ]
    avg_score = sum(scores) / len(scores) if scores else 0.0
    print(f"Ran {num_simulations} simulations for word '{word}'. Average score: {avg_score:.2f}")

    return avg_score

### Instantiate Agents and Group Chat
Now, create the agents and set up the group chat for them to collaborate.

In [ ]:
# --- YOUR CODE GOES HERE: Instantiate the agents ---

orchestrator_agent = AssistantAgent(
    name="Orchestrator", system_message=orchestrator_system_message, model_client=client
)

simulator_agent = AssistantAgent(
    name="Simulator",
    system_message=simulator_system_message,
    model_client=client,
    tools=[run_simulations_tool],
)

# --- END OF YOUR CODE ---

### Running the Agentic Workflow
Finally, create a group chat, add your agents, and initiate the task. The initial message to the Orchestrator will kick off the process. The communication between agents is a form of Agent-to-Agent (A2A) interaction.

In [ ]:
async def run_agentic_monte_carlo(verbose: bool = False):
    termination = TextMentionTermination("SUCCESSOR:")
    groupchat = RoundRobinGroupChat(
        [orchestrator_agent, simulator_agent],
        max_turns=50,  # Prevent infinite loops
        termination_condition=termination,
    )

    # Define the initial task for the Orchestrator
    initial_state = ""
    successor_states = puzzle.get_successor_states(initial_state)

    task = f"""
    The current word is empty. Find the best next letter to add.
    The possible successor words to evaluate are: {successor_states}.
    For each successor, ask the Aggregator to evaluate it with an appropriate number of simulations.
    Once all are evaluated, state the best one and then say TERMINATE.
    """

    result: TaskResult = await groupchat.run(task=task)

    # Parse the result to extract the chosen successor letter
    output = result.messages[-1].content
    if verbose:
        print("\n--- Full Agentic Monte Carlo Conversation ---")
        for msg in result.messages:
            print(msg.content)
    if output and "SUCCESSOR:" in output:
        chosen_letter = output.split("SUCCESSOR:")[-1]
        chosen_letter = chosen_letter.strip().replace(".", "").replace(",", "")

        print("\n--- Agentic Monte Carlo Result ---")
        print(f"Chosen next letter: '{chosen_letter}'")
    else:
        print("No valid successor found.")


# To run the async function in Jupyter
await run_agentic_monte_carlo()

### Reflection & Analysis (Part 2)

##### What worked well?
*Reflect on where the agentic decomposition was effective. Was the division of labor clear?*

##### What struggled?
*Note any challenges. Did agents misunderstand each other? Was the communication flow inefficient?*

##### Agentic vs. Conventional Implementation
*Compare the agentic implementation to the conventional one in Part 1. What are the advantages and disadvantages of the agentic approach in terms of code complexity, modularity, and extensibility?*

##### Communication Design (MCP/A2A)
*Describe your MCP and A2A design. For MCP, what was the schema for your `run_simulation` tool? For A2A, describe one key interaction (e.g., Orchestrator -> Aggregator). What was the purpose and what information was exchanged?*

### Summary and Next Steps
#### Key Takeaways
*   **Monte Carlo Search**: [Your insights about using random sampling for search]
*   **Heuristic Design**: [What you learned about guiding random search effectively]
*   **Agentic Decomposition**: [Your thoughts on breaking down algorithms into agent roles]

#### References
*   Lab 4 Overview Document
*   AutoGen Documentation: https://microsoft.github.io/autogen/
*   Russell, S., & Norvig, P. (2020). *Artificial Intelligence: A Modern Approach* (4th ed.).